In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import keras as kr

In [2]:
# DF de entrenamiento, con todas las variables y la columna tier completa
df_train: pd.DataFrame = pd.DataFrame(columns=("data", "tier"))
df_train.loc[0] = [50000, 1]
df_train.loc[1] = [49000, 2]
df_train.loc[2] = [40000, 2]
df_train.loc[3] = [39000, 3]
df_train.loc[4] = [30000, 3]
df_train.loc[5] = [21000, 3]
df_train.loc[6] = [20000, 4]
df_train.loc[7] = [11000, 4]
df_train.loc[8] = [10000, 5]


# DF de test, con todas las variables y la columna tier completa
df_test: pd.DataFrame = pd.DataFrame(columns=("data", "tier"))
df_test.loc[0] = [80000, 1]
df_test.loc[1] = [37000, 3]
df_test.loc[2] = [32000, 3]
df_test.loc[3] = [18000, 4]
df_test.loc[4] = [500, 5]
df_test.loc[5] = [200, 5]
df_test.loc[6] = [45000, 2]

## Preparación de datos

In [3]:
# Separar características y etiquetas

X_train: pd.DataFrame = df_train.drop('tier', axis=1)
y_train: pd.Series = df_train['tier']

X_test: pd.DataFrame = df_test.drop('tier', axis=1)
y_test: pd.Series = df_test['tier']

# Normalizar los datos
scaler: StandardScaler = StandardScaler()
X_train: pd.DataFrame = scaler.fit_transform(X_train)
X_test: pd.DataFrame = scaler.transform(X_test)

# Convertir etiquetas a categorías
y_train = kr.utils.to_categorical(y_train - 1, num_classes=5)
y_test = kr.utils.to_categorical(y_test - 1, num_classes=5)

## Definición del modelo

In [4]:
# Definir el modelo
model: kr.models.Sequential = kr.models.Sequential([
    kr.layers.Input(shape=(X_train.shape[1],)),
    kr.layers.Dense(64, activation='relu'),
    kr.layers.Dense(32, activation='relu'),
    kr.layers.Dense(5, activation='softmax')
])

# Compilar el modelo
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

## Entrenamiento y evaluación del modelo

In [5]:
# Aquí te he hecho un entrenamiento con unos parámetros a ojo, pero es solo 
# una prueba.

# Lo suyo sería que entrenaras con distintos epochs, validation_splits y 
# batch_size para encontrar el "punto dulce" de precisión del modelo.

# Entrenar el modelo
history = model.fit(
    X_train, y_train, epochs=100, validation_split=0.2, batch_size=1
)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.0357 - loss: 1.7137 - val_accuracy: 0.5000 - val_loss: 1.6288
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3845 - loss: 1.5740 - val_accuracy: 0.5000 - val_loss: 1.6253
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3908 - loss: 1.5648 - val_accuracy: 0.5000 - val_loss: 1.6288
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8262 - loss: 1.5250 - val_accuracy: 0.5000 - val_loss: 1.6250
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3283 - loss: 1.5153 - val_accuracy: 0.5000 - val_loss: 1.6355
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4220 - loss: 1.4638 - val_accuracy: 0.5000 - val_loss: 1.6476
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6512 - loss: 1.4655 - val_accuracy: 0.5000 - val_loss: 1.6639
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6512 - loss: 1.4439 - val_accuracy: 0.5000 - val_loss: 1.6799

## Predicción de tiers

In [6]:
# DF con datos pero sin valor de tiers
df_score: pd.DataFrame = pd.DataFrame(columns=("data",))

df_score.loc[0] = [100000]
df_score.loc[1] = [45500]
df_score.loc[2] = [27900]
df_score.loc[3] = [12000]
df_score.loc[4] = [150]

In [7]:
# Normalizar los datos de df_score
X_score = df_score.values
X_score = scaler.transform(X_score)

# Predecir los tiers
predictions = model.predict(X_score)
predicted_tiers = predictions.argmax(axis=1) + 1

# Añadir las predicciones a df_score
df_score['predicted_tier'] = predicted_tiers


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


c:\Users\pablo\Documents\GitHub\tf_tiers\env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [8]:
df_score

,data,predicted_tier
0,100000,1
1,45500,2
2,27900,3
3,12000,4
4,150,4
